
## Problem fomulation

In this project, we have some posts some of them are fake and some are real on raddit and as we know fake news has caused many social problems due to the raise of social network and its role in different domains such as politics. we are going to predict if a specific reddit post is fake news or not, by looking at its title. 

## input 
Text column / post

## output 
wheter if the post is Real or fake 

## challenges 
cleaning the data and make it ready by removing the whitespaces and special characters and so on..

## Data Mining function 
classification 

## impact 
detecting the falsy information


## optimal solution 
Logistic Regression by using Embedding and Random search dilvers pulic score of 0.85532

## Importing Libraries 

In [1]:
#useful imports
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
     
from google.colab import drive

## Loading The Data

In [2]:
#connect to drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
## loading the data
train_data = pd.read_csv("/content/drive/MyDrive/fake_reddit/xy_train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/fake_reddit/x_test.csv")

In [4]:
## The count of each class in the label column 
train_data['label'].value_counts()

0    32172
1    27596
2      232
Name: label, dtype: int64

In [5]:
## dropping the rows where the label == 2 as it is considered as outliers/noise
train_data= train_data[train_data.label != 2]

In [6]:
## checking one more time 
train_data['label'].value_counts()

0    32172
1    27596
Name: label, dtype: int64

## Cleaning and Preprocessing


In [7]:
# useful imports for word preprocessing 
import re
import pickle
import sklearn
import pandas as pd
import numpy as np
import holoviews as hv
import nltk 
from bokeh.io import output_notebook
output_notebook()

from pathlib import Path

# editing print settings in pandas and hvplot

pd.options.display.max_columns = 100
pd.options.display.max_rows = 300
pd.options.display.max_colwidth = 100
np.set_printoptions(threshold=2000)

In [8]:
pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# Imports
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import train_test_split, PredefinedSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
     


In [10]:
nltk.download('punkt')
nltk.download('stopwords')
#Choosing the steemmer language english
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english")) #stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def clean_text(text, for_embedding=False):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
# Apply stemming to convert tokens to their root form. This is a rule-based process of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary.
        
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words 
            
            ]

    text_clean = " ".join(words_filtered)
    return text_clean

In [12]:
# Clean texts for train dataset
train_data["text_clean"] = train_data.loc[train_data["text"].str.len() > 20, "text"]
train_data["text_clean"] = train_data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
train_data
     


,id,text,label,text_clean
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...,...
59995,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59996,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59997,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59998,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


In [13]:
## Clean texts for test data
test_data["text_clean"] = test_data.loc[test_data["text"].str.len() > 0, "text"]
test_data["text_clean"] = test_data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=False) if isinstance(x, str) else x
)
test_data

,id,text,text_clean
0,0,stargazer,stargaz
1,1,yeah,yeah
2,2,PD: Phoenix car thief gets instructions from YouTube video,pd phoenix car thief get instruct youtub video
3,3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility",trump accus iran one problem credibl
4,4,"""Believers"" - Hezbollah 2011",believ hezbollah
...,...,...,...
59146,59146,Bicycle taxi drivers of New Delhi,bicycl taxi driver new delhi
59147,59147,Trump blows up GOP's formula for winning House races,trump blow gop formula win hous race
59148,59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised",napoleon return exil island elba march colouris
59149,59149,Deep down he always wanted to be a ballet dancer,deep alway want ballet dancer


In [14]:
# Drop when clean text missing
train_data = train_data[(train_data["text_clean"] != "") & (train_data["text_clean"] != "null")]

train_data = train_data.dropna(axis="index", ).reset_index(drop=True)

In [15]:
train_data

,id,text,label,text_clean
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group friend began volunt homeless shelter neighbor protest see anoth person also need natur lik...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,british prime minist theresa may nerv attack former russian spi govern conclud high like russia ...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,goodyear releas kit allow ps brought heel https youtub com watch alxulk cg zwillc fish midatlant...
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,happi birthday bob barker price right host like rememb man said ave pet spay neuter fuckincorpor...
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,obama nation innoc cop unarm young black men die magic johnson jimbobshawobodob olymp athlet sho...
...,...,...,...,...
59753,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,finish sniper simo yh invas finland ussr color
59754,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,nigerian princ scam took kansa man year later get back
59755,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,safe smoke marijuana pregnanc surpris answer
59756,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,julius caesar upon realiz everyon room knife except bc


## splitting the data 

In [16]:
## split the data into X , y 
X = train_data['text_clean']
y = train_data['label']

X_test = test_data['text_clean']

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, train_size=0.8)
split_index = [-1 if i in X_train.index else 0 for i in X.index]
pds = PredefinedSplit(test_fold = split_index)

In [18]:
## Previewing The current shapes
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(47806,)
(47806,)
(11952,)
(11952,)


## Trials vectorizer analyzer="word"

### First_Trial 🥇
* classifier : Logestic Regresstion
* Search : Random search CV

In [19]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l1")), ("logistic", LogisticRegression())])

params = {
    "tfidf__ngram_range": [(1, 3),(1, 2)],
        # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5],
        # tfidf__ngram_range points to tfidf ->max_df

    "tfidf__min_df": [5],
        # tfidf__ngram_range points to tfidf ->min_df

}
#Search about best values to hyperparameters
pipe_lg_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_lg_clf.fit(X, y)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 2 is smaller than n_iter=3. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


CPU times: user 4.04 s, sys: 1.63 s, total: 5.67 s
Wall time: 17.7 s


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(norm='l1')),
                                             ('logistic',
                                              LogisticRegression())]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'tfidf__max_df': [0.5],
                                        'tfidf__min_df': [5],
                                        'tfidf__ngram_range': [(1, 3), (1, 2)]},
                   scoring='roc_auc')

In [20]:
print(pipe_lg_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 5, 'tfidf__max_df': 0.5}


In [21]:
## validation
pipe_lg_clf.score(X_val, y_val)

0.8742965766648668

In [23]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = test_data['id']

submission['label'] = pipe_lg_clf.predict_proba(X_test)[:,1]

submission.to_csv('/content/drive/MyDrive/fake_reddit/Trail_1.csv', index=False)

## Score : 0.81155 

**Observations:**  I have found that Logistic Regression is doing pretty well job at text analyzing ... as it doesn't require too much computitonal time and it's fairly easy to be understood

**Plans:** Let's Try to change the classifer we may get a better results  🧟

### Second_Trial 🥈
* classifier : xgboosting
* Search : Random search CV

In [25]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("XGB", XGBClassifier())])

# define parameter space to test
params = {
      "tfidf__ngram_range": [(1, 2)],
      "tfidf__max_df": [0.5],    
      "tfidf__min_df": [5,6,7],
      "XGB__learning_rate":[0.1, 0.25, 0.01, 0.51],
      "XGB__n_estimators":[200, 300, 400]
}

#Search about best values to hyperparameters
pipe_XGB_clf = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_XGB_clf.fit(X, y)

CPU times: user 3min 2s, sys: 589 ms, total: 3min 3s
Wall time: 6min 6s


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('XGB',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            ga...
                                                            min_child_weight=None,
                                                            missing=nan,
                                                            monotone_constraints=None,
                                                            n_estimators=100,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            predictor=None,
                                                            random_state=None, ...))]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'XGB__learning_rate': [0.1, 0.25, 0.01,
                                                               0.51],
                                        'XGB__n_estimators': [200, 300, 400],
                                        'tfidf__max_df': [0.5],
                                        'tfidf__min_df': [5, 6, 7],
                                        'tfidf__ngram_range': [(1, 2)]},
                   scoring='roc_auc')

In [26]:
print(pipe_XGB_clf.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 7, 'tfidf__max_df': 0.5, 'XGB__n_estimators': 300, 'XGB__learning_rate': 0.1}


In [27]:
# validation
pipe_XGB_clf.score(X_val, y_val)

0.8800485413879343

In [28]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = test_data['id']

submission['label'] = pipe_XGB_clf.predict_proba(X_test)[:,1]
submission.to_csv('/content/drive/MyDrive/fake_reddit/Trail_2.csv', index=False)

## Score: 0.78612 ☹

* **Observations** : in comparison between the two pervious Models it's obvious that LOgestic regresstion Is a better option... 
* **Plans** : Trying  LOgestic Regresstion one more Time but this Time by using Grid search in order to go through all the possible comibnations and to be grantuded to find out the optimal hyperparameters.. 

### Third_Trial 🥉
* classifier : Logestic Regresstion 
* Search : Grid search CV

In [29]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="word", norm="l2")), ("logistic", LogisticRegression())])

# define parameter space to test
params = {
    "tfidf__ngram_range": [(1, 3)],
    # tfidf__ngram_range points to tfidf ->ngram_range

    "tfidf__max_df": [0.5],
    # tfidf__ngram_range points to tfidf ->max_df
      
    "tfidf__min_df": [5],
      # tfidf__ngram_range points to tfidf ->min_df

    "logistic__penalty":['l1', 'l2']
    # logistic__penalty points to  logistic->penalty
}

#Search about best values to hyperparameters
pipe_lg_clf = GridSearchCV(pipe, params, n_jobs= -1, scoring="roc_auc", cv = pds, verbose=1)
pipe_lg_clf.fit(X, y)


Fitting 1 folds for each of 2 candidates, totalling 2 fits


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1 fits failed out of a total of 2.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "

CPU times: user 6.42 s, sys: 2.81 s, total: 9.23 s
Wall time: 18.3 s


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('logistic', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'logistic__penalty': ['l1', 'l2'],
                         'tfidf__max_df': [0.5], 'tfidf__min_df': [5],
                         'tfidf__ngram_range': [(1, 3)]},
             scoring='roc_auc', verbose=1)

In [30]:
print(pipe_lg_clf.best_params_)

{'logistic__penalty': 'l2', 'tfidf__max_df': 0.5, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 3)}


In [31]:
## validation
pipe_lg_clf.score(X_val, y_val)

0.9235346077207722

In [32]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = test_data['id']

submission['label'] = pipe_lg_clf.predict_proba(X_test)[:,1]
submission.to_csv('/content/drive/MyDrive/fake_reddit/Trail_3.csv', index=False)

## Score : 0.83756

## Summary For the previous Three trials 🥇 🥈 🥉

| classifer           | Search | Accuracy |
|---------------------|--------|----------|
| Logistic Regression | Random | 0.81155  |
| XGBoosting          | Random | 0.78612  |
| Logistic Regression | Grid   | 0.83756  |

* **Thoughts and conclustion** : Using Logistic Regression and Grid search delived the best results so far ....
* **Plans** : Testing out anothe preprocessing technique so we may get a better results this time ...

## Different preprocessing using embedding 🙂


In [19]:

nltk.download('punkt')
nltk.download('stopwords')
#Choosing the steemmer language english
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words("english")) #stopwords


def clean_text(text, for_embedding=True):
    """ steps:
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming, lowering and punctuation / stop words removal
        words_filtered = word_tokens
    else:
        words_filtered = [
# Apply stemming to convert tokens to their root form. This is a rule-based process of word form conversion where word-suffixes are truncated irrespective of whether the root word is an actual word in the language dictionary.
        
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words 
            
            ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# Clean texts for train dataset
train_data["text_clean"] = train_data.loc[train_data["text"].str.len() > 20, "text"]
train_data["text_clean"] = train_data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=True) if isinstance(x, str) else x
)
train_data

,id,text,label,text_clean
0,265723,"A group of friends began to volunteer at a homeless shelter after their neighbors protested. ""Se...",0,group of friends began to volunteer at homeless shelter after their neighbors protested . Seeing...
1,284269,"British Prime Minister @Theresa_May on Nerve Attack on Former Russian Spy: ""The government has c...",0,British Prime Minister Theresa May on Nerve Attack on Former Russian Spy The government has conc...
2,207715,"In 1961, Goodyear released a kit that allows PS2s to be brought to heel. https://m.youtube.com/w...",0,"In , Goodyear released kit that allows PS to be brought to heel . https youtube com watch ALXulk..."
3,551106,"Happy Birthday, Bob Barker! The Price Is Right Host on How He'd Like to Be Remembered | ""As the ...",0,"Happy Birthday , Bob Barker ! The Price Is Right Host on How He Like to Be Remembered As the man..."
4,8584,"Obama to Nation: 聙""Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Jo...",0,Obama to Nation Innocent Cops and Unarmed Young Black Men Should Not be Dying Before Magic Johns...
...,...,...,...,...
59753,70046,"Finish Sniper Simo H盲yh盲 during the invasion of Finland by the USSR (1939, colorized)",0,"Finish Sniper Simo yh during the invasion of Finland by the USSR , colorized"
59754,189377,"Nigerian Prince Scam took $110K from Kansas man; 10 years later, he's getting it back",1,"Nigerian Prince Scam took from Kansas man years later , he getting it back"
59755,93486,Is It Safe To Smoke Marijuana During Pregnancy? You鈥檇 Be Surprised Of The Answer | no,0,Is It Safe To Smoke Marijuana During Pregnancy ? You Be Surprised Of The Answer no
59756,140950,Julius Caesar upon realizing that everyone in the room has a knife except him (44 bc),0,Julius Caesar upon realizing that everyone in the room has knife except him bc


In [21]:
#Clean text for test data
test_data["text_clean"] = test_data.loc[test_data["text"].str.len() > 0, "text"]
test_data["text_clean"] = test_data["text_clean"].map(
    lambda x: clean_text(x, for_embedding=True) if isinstance(x, str) else x
)
test_data

,id,text,text_clean
0,0,stargazer,stargazer
1,1,yeah,yeah
2,2,PD: Phoenix car thief gets instructions from YouTube video,PD Phoenix car thief gets instructions from YouTube video
3,3,"As Trump Accuses Iran, He Has One Problem: His Own Credibility","As Trump Accuses Iran , He Has One Problem His Own Credibility"
4,4,"""Believers"" - Hezbollah 2011",Believers Hezbollah
...,...,...,...
59146,59146,Bicycle taxi drivers of New Delhi,Bicycle taxi drivers of New Delhi
59147,59147,Trump blows up GOP's formula for winning House races,Trump blows up GOP formula for winning House races
59148,59148,"Napoleon returns from his exile on the island of Elba. (March 1815), Colourised","Napoleon returns from his exile on the island of Elba . March , Colourised"
59149,59149,Deep down he always wanted to be a ballet dancer,Deep down he always wanted to be ballet dancer


In [22]:
# Drop when any of x missing
train_data = train_data[(train_data["text_clean"] != "") & (train_data["text_clean"] != "null")]

train_data = train_data.dropna(axis="index", ).reset_index(drop=True)

## splitting the Data

In [23]:
## split the data into X , y 
X = train_data['text_clean']
y = train_data['label']

X_test = test_data['text_clean']

In [24]:
# Further split the original training set to a train and a validation set
X_train2, X_val, y_train2, y_val = train_test_split(
    X, y , train_size = 0.7, stratify = y , random_state = 4200)

# Create a list where train data indices are -1 and validation data indices are 0
# X_train2 (new training set), X_train
split_index = [-1 if x in X_train2.index else 0 for x in X_train.index]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

## Fourth Trial: ⭐ ⭐ ⭐ ⭐ ⭐ [The Optimal Solution]
* Classifier: Logistic Regression 
* Search : Radnom Search 

In [39]:
pipe = Pipeline(
    [
                 ("tfidf", TfidfVectorizer()), 
                 ("LOG", LogisticRegression())])
                 

# define parameter space to test 
params = {
    "tfidf__ngram_range": [(1, 2), (1, 3)],
    "tfidf__max_df": np.arange(0.3, 0.8),
    "tfidf__min_df": np.arange(5, 100),

}
pipe_lg_3 =RandomizedSearchCV(
    pipe, params, n_jobs=2, scoring="roc_auc", cv=pds, n_iter=5)
pipe_lg_3.fit(X, y)

RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0, -1])),
                   estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                             ('LOG', LogisticRegression())]),
                   n_iter=5, n_jobs=2,
                   param_distributions={'tfidf__max_df': array([0.3]),
                                        'tfidf__min_df': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'tfidf__ngram_range': [(1, 2), (1, 3)]},
                   scoring='roc_auc')

In [40]:
print(pipe_lg_3.best_params_)

{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 22, 'tfidf__max_df': 0.3}


In [41]:
## validation
pipe_lg_3.score(X_val, y_val)

0.9258595684238475

In [42]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = test_data['id']

submission['label'] = pipe_lg_3.predict_proba(X_test)[:,1]
submission.to_csv('/content/drive/MyDrive/fake_reddit/Trail_4.csv', index=False)

## Score : 0.85532 🆒

 **Observations** : By using embedding it's obvious that it's the optimal preprocessing used so far 

 **plans** : let's test out another calssifer which is SVM...

## Fifth Trial : 😮
* classifer: XGBoosting 
* search: Random search

In [26]:
%%time
# feature creation and modelling in a single function
pipe = Pipeline([("tfidf", TfidfVectorizer(analyzer="char", norm="l2")), ("XGB", XGBClassifier())])

# define parameter space to test 
params = {
    "tfidf__ngram_range": [(1, 2)],

    "tfidf__max_df": [0.5,1,0.6, 0.9],
      
    "tfidf__min_df": [5, 4, 6 ,8],

}

#Search about best values to hyperparameters
pipe_XGB_clf2 = RandomizedSearchCV(pipe, params, n_jobs=-1, scoring="roc_auc", cv = pds, n_iter=3)
pipe_XGB_clf2.fit(X, y)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
1 fits failed out of a total of 3.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.9/dist-packages/jo

CPU times: user 6min 33s, sys: 913 ms, total: 6min 34s
Wall time: 7min 59s


RandomizedSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0, -1])),
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(analyzer='char')),
                                             ('XGB',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature...
                                                            max_delta_step=None,
                                                            max_depth=None,
                                                            max_leaves=None,
                                                            min_child_weight=None,
                                                            missing=nan,
                                                            monotone_constraints=None,
                                                            n_estimators=100,
                                                            n_jobs=None,
                                                            num_parallel_tree=None,
                                                            predictor=None,
                                                            random_state=None, ...))]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'tfidf__max_df': [0.5, 1, 0.6, 0.9],
                                        'tfidf__min_df': [5, 4, 6, 8],
                                        'tfidf__ngram_range': [(1, 2)]},
                   scoring='roc_auc')

In [27]:
print(pipe_XGB_clf2.best_params_)


{'tfidf__ngram_range': (1, 2), 'tfidf__min_df': 8, 'tfidf__max_df': 0.9}


In [28]:
## validation
pipe_XGB_clf2.score(X_val, y_val)

0.9474096281347505

In [29]:
#Write the result in csv file

submission = pd.DataFrame()
submission['id'] = test_data['id']

submission['label'] = pipe_XGB_clf2.predict_proba(X_test)[:,1]
submission.to_csv('/content/drive/MyDrive/fake_reddit/Trail_5.csv', index=False)

## Score : 0.77701

## ❓❓ Questions ❓❓


 ###  🌈 What is the difference between Character n-gram and Word n-gram ❓ Which one tends to suffer more from the OOV issue ❓

 * Character n-gram : split the text into words.    
 * Word n-gram  : split the text into set of Characters.  

 OOV stands for out of vocaulary 

 and word n-gram tends to suffer from the problem of OOV more  


## 🌈 What is the difference between stop word removal and stemming ❓ Are these techniques language-dependent ❓

* **Stop Words** : words that appear most frequently in all of the documents . and we could remove them . examples (is, the, and , or ,....etc)


* **Stemming** : Stemming is a technique used to reduce an inflected word down to its word stem. For example, the words “programming,” “programmer,” and “programs” can all be reduced down to the common word stem “program.”

* Yes these techniques are Language dependenat because each language has it's own stop words and stems .



## 🌈 Is tokenization techniques language dependent❓  Why ❓
* Tokenization is a way of separating a piece of text into smaller units called tokens. Here, tokens can be either words, characters, or subwords.

* it is language dependent as it should know the meaning of each single word to split it depend on language.

## 🌈 What is the difference between count vectorizer and tf-idf vectorizer ❓ Would it be feasible to use all possible n-grams ❓ If not, how should you select them ❓
 * **Count Vectorizer** : simply counts the number of times a word appears in a document (using a bag-of-words approach),

 * **TF-IDF Vectorizer** : takes into account not only how many times a word appears in a document but also how important that word is to the whole corpus.

 No it's to feasible to use all possible n-grams.


#### **we select them  based on the business problem and data:** 

 * it's better to choose `IF-IDF Vectorizer` when the data is longer and contains more unique words.

 * while it's better to choose `Count Vectorizer` when the data is shorter and doesnt have many unique words.

